In various figures I display information about the time averaged advective heat transport in and out of regions, as well as the temperature tendency and surface heat fluxes. These are displayed for the entire continental shelf, the Rodd Sea shelf region, and the West Antarctc shelf region. Such computations are outlined in this notebook. In this study we had to approximate suface heat fluxes as the residual of other heat budget terms (a balance we sconfirmed held for a re-run year of the MW45 experiment) due to an erroneous formulation of the net_sfc_heating diagnostic being output from the ACCESS-OM2 suite. This has been corrected for future runs.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import xarray as xr
import numpy as np
### establish link to python functions
import sys
sys.path.append('/home/156/rm2389/Honours_Thesis/python_functions/')
### import required python functions
from masking import shelf_mask_isobath, fill_land
from spatial_averaging import month_weights
import cosima_cookbook as cc
from dask.distributed import Client
client = Client('tcp://localhost:8786')
client

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 6 Cores: 6 Memory: 18.00 GB


In [2]:
db = '/g/data/v45/rm2389/databases/freshwater_experiments.db' # database path
session = cc.database.create_session(db)
control='01deg_jra55v13_ryf8485_spinup6'
rcp45='01deg_jra55v13_ryf8485_freshRCP45'
rcp85='01deg_jra55v13_ryf8485_freshRCP85'

*Region definitions*

Continental shelf: poleward of the 1000 m isobath

Ross Sea shelf: continental shelf between -157 and -198 longitude

West Antarctic shelf: continental shelf between -63 and -157 longitude

*Variables*

`temp_xflux_adv`: `cp*rho*dzt*dyt*u*temp`

`heat_trans_across_isobath`: computed offline using outputs `temp_xflux_adv`  and `temp_yflux_adv` sorted onto the 1000m isobath as in `computing_cross_1000m_isobath_transports.ipynb` and saved.


tenperature snapshots (either from restarts or daily data) are used to approximate the temp tendency over the analysis period.

I also computed volume fluxes across boundaries to guide interpretation of 0C referenced heat budgets.

In [3]:
timeslice = slice('1937-06', '1947-05')
latslice = slice(-90,-59)
dyt = cc.querying.getvar(control, 'dyt', session, ncfile = 'ocean_grid.nc', n=1)
dxt = cc.querying.getvar(control, 'dxt', session, ncfile = 'ocean_grid.nc', n=1)
dxu = cc.querying.getvar(control, 'dxu', session, ncfile = 'ocean_grid.nc', n=1)
yt_ocean = dyt.yt_ocean
xt_ocean = dyt.xt_ocean
yu_ocean = dxu.yu_ocean
xu_ocean = dxu.xu_ocean

### Control

In [16]:
## load the isobath transports
heat_trans_across_isobath = xr.open_dataset( '/g/data/v45/rm2389/Isobath_Transport/control/heat_budget_monthly_data/heat_trans_across_isobath_1937-06-1947-05.nc')
heat_trans_across_isobath = - heat_trans_across_isobath.heat_trans_across_isobath.sel(time = timeslice)

## load zonal heat and volume flux for lateral boundaries
temp_xflux_adv = cc.querying.getvar(control, 'temp_xflux_adv',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')
xflux_adv = cc.querying.getvar(control, 'tx_trans',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')

## shelf masking function require dimensions of xt/yt so will temporarily replace dimension xu with xt 
## to mask and then switch back
temp_xflux_adv.coords['xu_ocean'] = xt_ocean.values
temp_xflux_adv = temp_xflux_adv.rename({'xu_ocean':'xt_ocean'})
xflux_adv.coords['xu_ocean'] = xt_ocean.values
xflux_adv = xflux_adv.rename({'xu_ocean':'xt_ocean'})

## mask for the continental shelf
temp_xflux_adv = temp_xflux_adv.sel(time = timeslice)
xflux_adv = xflux_adv.sel(time = timeslice)
temp_xflux_adv_shelf, shelf_mask = shelf_mask_isobath(temp_xflux_adv)
xflux_adv_shelf, shelf_mask = shelf_mask_isobath(xflux_adv)

## re-establish correct x dimension for clarity
temp_xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
temp_xflux_adv_shelf = temp_xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})
xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
xflux_adv_shelf = xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})

In [20]:
## temperature snapshots for calculating the temp  tendency
# start
snapshot_start ='/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_freshRCP45/start_end_snapshots/restart416_pot_temp.nc'
temp_start = xr.open_dataset(snapshot_start, chunks = {'st_ocean':5, 'yt_ocean':300, 'xt_ocean':400})
temp_start = temp_start.temp
temp_start_shelf, shelf_mask = shelf_mask_isobath(temp_start)
# end
temp_daily = cc.querying.getvar(control, 'temp', session, ncfile='rregionsouthern_ocean_daily_temp.nc')
temp_daily = temp_daily.isel(yt_ocean_sub01 = slice(10,None)) ## get rid of 0s
temp_daily = temp_daily.rename({'yt_ocean_sub01':'yt_ocean', 'xt_ocean_sub01':'xt_ocean'})
temp_daily_end = temp_daily.sel(time = '1947-05-31', method = 'nearest') - 273.15
temp_daily_end_shelf, shelf_mask = shelf_mask_isobath(temp_daily_end)
temp_daily_end_shelf = temp_daily_end_shelf.squeeze(dim = 'time')
## volume
dzt = cc.querying.getvar(control,'dzt',session,ncfile= 'ocean.nc')
dzt_end = dzt.sel(time = slice('1947-05','1947-06')).mean(dim = 'time')
dzt_start = dzt.sel(time = slice('1937-05','1937-06')).mean(dim = 'time')
volume_start =  dzt_start * dyt * dxt
volume_end =  dzt_end * dyt * dxt
volume_start_shelf, shelf_mask = shelf_mask_isobath(volume_start)
volume_end_shelf, shelf_mask = shelf_mask_isobath(volume_end)

In [19]:
time = temp_xflux_adv_shelf.time
heat_trans_across_isobath.coords['time'] = time
weights = xr.DataArray(month_weights(6,5,10), coords = [time], dims = 'time')
weights = weights.chunk({'time':1})

In [23]:
########################
#######FULL SHELF#######
########################
# advective heat transport across isobath (no need for volume on this one)
isobath_heat_flux_summed = heat_trans_across_isobath.sum(dim = 'st_ocean').sum(dim = 'isobath_index')
isobath_heat_flux_mean = (isobath_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
# temperature tendency
# start
volume_start_shelf_sum = volume_start_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_start_shelf = (temp_start_shelf * volume_start_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_start = temp_vol_start_shelf/volume_start_shelf_sum
heat_start_shelf = temp_vol_start_shelf * 3992.10322329649 * 1035 
# end
volume_end_shelf_sum = volume_end_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_end_shelf = (temp_daily_end_shelf * volume_end_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_end = temp_vol_end_shelf/volume_end_shelf_sum
heat_end_shelf = temp_vol_end_shelf * 3992.10322329649 * 1035 
seconds = 60 * 60 * 24 * 365 * 10 # seconds in 10 year
heat_content_change = (heat_end_shelf - heat_start_shelf)/seconds * 1e-12

print('Full shelf stats (control)')
print('Heat flux across the isobath: %3.2f TW'%isobath_heat_flux_mean)
print('Change in heat content: %3.2f TW'%heat_content_change)
print('Surf (change in heat minus sum of adv): %3.2f'%(heat_content_change-isobath_heat_flux_mean))


Full shelf stats (control)
Heat flux across the isobath: 42.24 TW
Change in heat content: 0.17 TW
Surf (change in heat minus sum of adv): -42.07


In [54]:
## alter cause we have snapshots for the end of the perturbed runs
def compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, name, run):
    # advective fluxes
    western_heat_flux = temp_xflux_adv_shelf.sel(xu_ocean = lon_west, method = 'nearest')
    eastern_heat_flux = -temp_xflux_adv_shelf.sel(xu_ocean = lon_east, method = 'nearest')
    western_flux = xflux_adv_shelf.sel(xu_ocean = lon_west, method = 'nearest')
    eastern_flux = -xflux_adv_shelf.sel(xu_ocean = lon_east, method = 'nearest')
    isobath_heat_flux = heat_trans_across_isobath.sel(isobath_index = slice(isobath_start, isobath_end))
    western_heat_flux_summed = western_heat_flux.sum(dim = 'st_ocean').sum(dim = 'yt_ocean')
    eastern_heat_flux_summed = eastern_heat_flux.sum(dim = 'st_ocean').sum(dim = 'yt_ocean')
    western_flux_summed = western_flux.sum(dim = 'st_ocean').sum(dim = 'yt_ocean')
    eastern_flux_summed = eastern_flux.sum(dim = 'st_ocean').sum(dim = 'yt_ocean')
    isobath_heat_flux_summed = isobath_heat_flux.sum(dim = 'st_ocean').sum(dim = 'isobath_index')
    western_heat_flux_mean = (western_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
    eastern_heat_flux_mean = (eastern_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
    isobath_heat_flux_mean = (isobath_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
    western_flux_mean = (western_flux_summed * weights).mean(dim = 'time').values * 1/1035 * 1e-6
    eastern_flux_mean = (eastern_flux_summed * weights).mean(dim = 'time').values * 1/1035 * 1e-6
    # temp tendency
    #start
    region_temp_start = temp_start_shelf.sel(xt_ocean = slice(lon_west,lon_east))
    region_vol_start = volume_start_shelf.sel(xt_ocean = slice(lon_west,lon_east))
    region_vol_sum_start = region_vol_start.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
    temp_vol_start = (region_temp_start* region_vol_start).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
    vol_ave_temp_start = temp_vol_start/region_vol_sum_start
    heat_start = temp_vol_start * 3992.10322329649 * 1035 
    # end
    if run == 'control':
        region_temp_end = temp_daily_end_shelf.sel(xt_ocean = slice(lon_west,lon_east))
    else:
        region_temp_end = temp_end_shelf.sel(xt_ocean = slice(lon_west,lon_east))

    region_vol_end = volume_end_shelf.sel(xt_ocean = slice(lon_west,lon_east))
    region_vol_sum_end = region_vol_end.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
    temp_vol_end = (region_temp_end* region_vol_end).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
    vol_ave_temp_end = temp_vol_end/region_vol_sum_end
    heat_end = temp_vol_end * 3992.10322329649 * 1035 
    seconds = 60 * 60 * 24 * 365 * 10 
    heat_content_change = (heat_end - heat_start)/seconds * 1e-12
    print(name+' shelf ('+run+')')
    print('Heat flux across the isobath: %2.1f TW'%isobath_heat_flux_mean)
    print('Heat flux across the western boundary: %2.1f TW'%western_heat_flux_mean)
    print('Heat flux across the eastern boundary: %2.1f TW'%eastern_heat_flux_mean)
    print('Change in heat content: %2.1f TW'%heat_content_change)
    print('Surf (change in heat minus sum of adv): %2.1f'%(heat_content_change-(isobath_heat_flux_mean+western_heat_flux_mean+eastern_heat_flux_mean)))
    print('Volume flux across the west boundary: %2.1f Sv'%western_flux_mean)
    print('Volume flux across the east boundary: %2.1f Sv'%eastern_flux_mean)

In [37]:
########################
#######ROSS SHELF#######
########################
lon_west = -198
lon_east = -157
isobath_start = 1307
isobath_end = 2055
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'Ross', 'control')

Ross shelf (control)
Heat flux across the isobath: 11.9 TW
Heat flux across the western boundary: 3.0 TW
Heat flux across the eastern boundary: -3.6 TW
Change in heat content: -0.0 TW
Surf (change in heat minus sum of adv): -11.4
Volume flux across the west boundary: -0.6 Sv
Volume flux across the east boundary: 0.6 Sv


In [38]:
##################################
#######WEST ANTARCTIC SHELF#######
##################################
lon_west = -157
lon_east = -63
isobath_start = 2056
isobath_end = 3373
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'West Antarctic', 'control')

West Antarctic shelf (control)
Heat flux across the isobath: 5.8 TW
Heat flux across the western boundary: 3.6 TW
Heat flux across the eastern boundary: -0.4 TW
Change in heat content: 0.2 TW
Surf (change in heat minus sum of adv): -8.8
Volume flux across the west boundary: -0.6 Sv
Volume flux across the east boundary: -0.0 Sv


### MW45

In [39]:
## load the isobath transports
heat_trans_across_isobath = xr.open_dataset( '/g/data/v45/rm2389/Isobath_Transport/rcp45/heat_trans_across_isobath_1937-06-1947-05.nc')
heat_trans_across_isobath = - heat_trans_across_isobath.heat_trans_across_isobath.sel(time = timeslice)

## load zonal heat and volume flux for lateral boundaries
temp_xflux_adv = cc.querying.getvar(rcp45, 'temp_xflux_adv',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')
xflux_adv = cc.querying.getvar(rcp45, 'tx_trans',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')

## shelf masking function require dimensions of xt/yt so will temporarily replace dimension xu with xt 
## to mask and then switch back
temp_xflux_adv.coords['xu_ocean'] = xt_ocean.values
temp_xflux_adv = temp_xflux_adv.rename({'xu_ocean':'xt_ocean'})
xflux_adv.coords['xu_ocean'] = xt_ocean.values
xflux_adv = xflux_adv.rename({'xu_ocean':'xt_ocean'})

## mask for the continental shelf
temp_xflux_adv = temp_xflux_adv.sel(time = timeslice)
xflux_adv = xflux_adv.sel(time = timeslice)
temp_xflux_adv_shelf, shelf_mask = shelf_mask_isobath(temp_xflux_adv)
xflux_adv_shelf, shelf_mask = shelf_mask_isobath(xflux_adv)

## re-establish correct x dimension for clarity
temp_xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
temp_xflux_adv_shelf = temp_xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})
xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
xflux_adv_shelf = xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})

In [48]:
temp_xflux_adv_shelf = temp_xflux_adv_shelf.isel(time = slice(0,120))
xflux_adv_shelf = xflux_adv_shelf.isel(time = slice(0,120))

In [42]:
## temperature snapshots for calculating the temp  tendency
snapshot_start = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_freshRCP45/start_end_snapshots/restart416_pot_temp.nc'
snapshot_end = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_freshRCP45/start_end_snapshots/restart536_pot_temp.nc'
temp_start = xr.open_dataset(snapshot_start, chunks = {'st_ocean':5, 'yt_ocean':300, 'xt_ocean':400})
temp_end = xr.open_dataset(snapshot_end, chunks = {'st_ocean':5, 'yt_ocean':300, 'xt_ocean':400})
temp_start = temp_start.temp
temp_end = temp_end.temp
temp_start_shelf, shelf_mask = shelf_mask_isobath(temp_start)
temp_end_shelf, shelf_mask = shelf_mask_isobath(temp_end)
## volume
dzt = cc.querying.getvar(control,'dzt',session,ncfile= 'ocean.nc')
dzt_end = dzt.sel(time = slice('1947-05','1947-06')).mean(dim = 'time')
dzt_start = dzt.sel(time = slice('1937-05','1937-06')).mean(dim = 'time')
volume_start =  dzt_start * dyt * dxt
volume_end =  dzt_end * dyt * dxt
volume_start_shelf, shelf_mask = shelf_mask_isobath(volume_start)
volume_end_shelf, shelf_mask = shelf_mask_isobath(volume_end)

In [49]:
time = temp_xflux_adv_shelf.time
heat_trans_across_isobath.coords['time'] = time
weights = xr.DataArray(month_weights(6,5,10), coords = [time], dims = 'time')
weights = weights.chunk({'time':1})

In [50]:
########################
#######FULL SHELF#######
########################
# advective heat transport across isobath (no need for volume on this one)
isobath_heat_flux_summed = heat_trans_across_isobath.sum(dim = 'st_ocean').sum(dim = 'isobath_index')
isobath_heat_flux_mean = (isobath_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
# temperature tendency
# start
volume_start_shelf_sum = volume_start_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_start_shelf = (temp_start_shelf * volume_start_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_start = temp_vol_start_shelf/volume_start_shelf_sum
heat_start_shelf = temp_vol_start_shelf * 3992.10322329649 * 1035 
# end
volume_end_shelf_sum = volume_end_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_end_shelf = (temp_end_shelf * volume_end_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_end = temp_vol_end_shelf/volume_end_shelf_sum
heat_end_shelf = temp_vol_end_shelf * 3992.10322329649 * 1035 
seconds = 60 * 60 * 24 * 365 * 10 # seconds in 10 year
heat_content_change = (heat_end_shelf - heat_start_shelf)/seconds * 1e-12

print('Full shelf stats (mw45)')
print('Heat flux across the isobath: %3.2f TW'%isobath_heat_flux_mean)
print('Change in heat content: %3.2f TW'%heat_content_change)
print('Surf (change in heat minus sum of adv): %3.2f'%(heat_content_change-isobath_heat_flux_mean))


Full shelf stats (mw45)
Heat flux across the isobath: 30.34 TW
Change in heat content: -0.43 TW
Surf (change in heat minus sum of adv): -30.77


We take these heat fluxes as anomalies from the control:

Heat flux across the isobath: -11.9 TW

Change in heat content: -0.6 TW

Surf (change in heat minus sum of adv): +11.3 TW


In [55]:
########################
#######ROSS SHELF#######
########################
lon_west = -198
lon_east = -157
isobath_start = 1307
isobath_end = 2055
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'Ross', 'rcp45')

Ross shelf (rcp45)
Heat flux across the isobath: 4.0 TW
Heat flux across the western boundary: 7.2 TW
Heat flux across the eastern boundary: -4.9 TW
Change in heat content: 0.9 TW
Surf (change in heat minus sum of adv): -5.5
Volume flux across the west boundary: -1.1 Sv
Volume flux across the east boundary: 0.8 Sv


well shoot it seems I put some incorrect numbers in the paper, must revise.

In [57]:
##################################
#######WEST ANTARCTIC SHELF#######
##################################
lon_west = -157
lon_east = -63
isobath_start = 2056
isobath_end = 3373
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'West Antarctic', 'rcp45')

West Antarctic shelf (rcp45)
Heat flux across the isobath: 6.5 TW
Heat flux across the western boundary: 4.9 TW
Heat flux across the eastern boundary: -3.2 TW
Change in heat content: -0.2 TW
Surf (change in heat minus sum of adv): -8.4
Volume flux across the west boundary: -0.8 Sv
Volume flux across the east boundary: 0.5 Sv


### MW85

In [60]:
## load the isobath transports
heat_trans_across_isobath = xr.open_dataset( '/g/data/v45/rm2389/Isobath_Transport/rcp85/heat_trans_across_isobath_1937-06-1947-05.nc')
heat_trans_across_isobath = - heat_trans_across_isobath.heat_trans_across_isobath.sel(time = timeslice)

## load zonal heat and volume flux for lateral boundaries
temp_xflux_adv = cc.querying.getvar(rcp85, 'temp_xflux_adv',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')
xflux_adv = cc.querying.getvar(rcp85, 'tx_trans',session, ncfile = 'ocean.nc', start_time = '1937-01', end_time = '1947-12')

## shelf masking function require dimensions of xt/yt so will temporarily replace dimension xu with xt 
## to mask and then switch back
temp_xflux_adv.coords['xu_ocean'] = xt_ocean.values
temp_xflux_adv = temp_xflux_adv.rename({'xu_ocean':'xt_ocean'})
xflux_adv.coords['xu_ocean'] = xt_ocean.values
xflux_adv = xflux_adv.rename({'xu_ocean':'xt_ocean'})

## mask for the continental shelf
temp_xflux_adv = temp_xflux_adv.sel(time = timeslice)
xflux_adv = xflux_adv.sel(time = timeslice)
temp_xflux_adv_shelf, shelf_mask = shelf_mask_isobath(temp_xflux_adv)
xflux_adv_shelf, shelf_mask = shelf_mask_isobath(xflux_adv)

## re-establish correct x dimension for clarity
temp_xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
temp_xflux_adv_shelf = temp_xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})
xflux_adv_shelf.coords['xt_ocean'] = xu_ocean.values
xflux_adv_shelf = xflux_adv_shelf.rename({'xt_ocean':'xu_ocean'})

In [59]:
## temperature snapshots for calculating the temp  tendency
snapshot_start = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_freshRCP45/start_end_snapshots/restart416_pot_temp.nc'
snapshot_end = '/g/data3/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf8485_freshRCP85/start_end_snapshots/restart536_pot_temp.nc'
temp_start = xr.open_dataset(snapshot_start, chunks = {'st_ocean':5, 'yt_ocean':300, 'xt_ocean':400})
temp_end = xr.open_dataset(snapshot_end, chunks = {'st_ocean':5, 'yt_ocean':300, 'xt_ocean':400})
temp_start = temp_start.temp
temp_end = temp_end.temp
temp_start_shelf, shelf_mask = shelf_mask_isobath(temp_start)
temp_end_shelf, shelf_mask = shelf_mask_isobath(temp_end)
## volume
dzt = cc.querying.getvar(rcp85,'dzt',session,ncfile= 'ocean.nc')
dzt_end = dzt.sel(time = slice('1947-05','1947-06')).mean(dim = 'time')
dzt_start = dzt.sel(time = slice('1937-05','1937-06')).mean(dim = 'time')
volume_start =  dzt_start * dyt * dxt
volume_end =  dzt_end * dyt * dxt
volume_start_shelf, shelf_mask = shelf_mask_isobath(volume_start)
volume_end_shelf, shelf_mask = shelf_mask_isobath(volume_end)

In [61]:
time = temp_xflux_adv_shelf.time
heat_trans_across_isobath.coords['time'] = time
weights = xr.DataArray(month_weights(6,5,10), coords = [time], dims = 'time')
weights = weights.chunk({'time':1})

In [62]:
########################
#######FULL SHELF#######
########################
# advective heat transport across isobath (no need for volume on this one)
isobath_heat_flux_summed = heat_trans_across_isobath.sum(dim = 'st_ocean').sum(dim = 'isobath_index')
isobath_heat_flux_mean = (isobath_heat_flux_summed * weights).mean(dim = 'time').values * 1e-12
# temperature tendency
# start
volume_start_shelf_sum = volume_start_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_start_shelf = (temp_start_shelf * volume_start_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_start = temp_vol_start_shelf/volume_start_shelf_sum
heat_start_shelf = temp_vol_start_shelf * 3992.10322329649 * 1035 
# end
volume_end_shelf_sum = volume_end_shelf.sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
temp_vol_end_shelf = (temp_end_shelf * volume_end_shelf).sum(dim = 'st_ocean').sum(dim = 'xt_ocean').sum(dim = 'yt_ocean').values
vol_ave_temp_end = temp_vol_end_shelf/volume_end_shelf_sum
heat_end_shelf = temp_vol_end_shelf * 3992.10322329649 * 1035 
seconds = 60 * 60 * 24 * 365 * 10 # seconds in 10 year
heat_content_change = (heat_end_shelf - heat_start_shelf)/seconds * 1e-12

print('Full shelf stats (mw85)')
print('Heat flux across the isobath: %3.2f TW'%isobath_heat_flux_mean)
print('Change in heat content: %3.2f TW'%heat_content_change)
print('Surf (change in heat minus sum of adv): %3.2f'%(heat_content_change-isobath_heat_flux_mean))


Full shelf stats (mw85)
Heat flux across the isobath: 22.20 TW
Change in heat content: -0.50 TW
Surf (change in heat minus sum of adv): -22.70


Anomalies for figure:

Heat flux across the isobath: -20.1 TW

Change in heat content: 0.7 TW

Surf (change in heat minus sum of adv): +19.4 TW



In [63]:
########################
#######ROSS SHELF#######
########################
lon_west = -198
lon_east = -157
isobath_start = 1307
isobath_end = 2055
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'Ross', 'rcp85')

Ross shelf (rcp85)
Heat flux across the isobath: 1.0 TW
Heat flux across the western boundary: 11.2 TW
Heat flux across the eastern boundary: -8.1 TW
Change in heat content: 1.6 TW
Surf (change in heat minus sum of adv): -2.6
Volume flux across the west boundary: -1.9 Sv
Volume flux across the east boundary: 1.5 Sv


Well I need to redo my ross sea analysis and resubmit...

In [64]:
##################################
#######WEST ANTARCTIC SHELF#######
##################################
lon_west = -157
lon_east = -63
isobath_start = 2056
isobath_end = 3373
compute_region_heat_budget(lon_west, lon_east, isobath_start, isobath_end, 'West Antarctic', 'rcp85')

West Antarctic shelf (rcp85)
Heat flux across the isobath: 7.0 TW
Heat flux across the western boundary: 8.1 TW
Heat flux across the eastern boundary: -8.3 TW
Change in heat content: -2.6 TW
Surf (change in heat minus sum of adv): -9.5
Volume flux across the west boundary: -1.5 Sv
Volume flux across the east boundary: 1.5 Sv
